# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment to train two agents to play Tennis.  This is the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the [Getting Started](./README.md) section of the project README.md.

In [1]:
from unityagents import UnityEnvironment
import numpy as np

In [2]:
def print_obs(vec_obs, actions=None):
    header0 = ['racket', 'racket', 'racket', 'racket', 'ball', 'ball', 'ball', 'ball']
    header1 = ['x', 'y', 'vx', 'vy', 'x', 'y', 'vx', 'vy']
    player = ['Right 1', 'Left 2']
    for j, obs in enumerate(vec_obs):
        if actions is not None:
            print(actions[j])
        print(f'{player[j]:>8}', ''.join(map(lambda x: f"{x:>9}", header0[:4])), '|', ''.join(map(lambda x: f"{x:>9}", header0[-4:])))
        print(' '*8, ''.join(map(lambda x: f"{x:>9}", header1[:4])), '|', ''.join(map(lambda x: f"{x:>9}", header1[-4:])))
        for i in range(len(obs)):
            print(f'{i:>8}', ''.join(map(lambda x: f"{x:>9.4f}", obs[i][:4])), '|', ''.join(map(lambda x: f"{x:>9.4f}", obs[i][-4:])))
        print('')

In [3]:
test_actions = np.array([
    [[0.0, 0.], [0.1, 0.]],
    [[-0.1, 0.6], [0.0, 0.6]],
    [[-0.1, 0.6], [0.1, 0.6]],
    [[0.0, 0.6], [-0.1, 0.6]],
    [[0.1, 0.6], [0.0, 0.6]],
    [[0.1, 0.6], [-0.1, 0.6]]])

In [3]:
test_actions = np.array([
    [[0.0, 0.6], [0.0, 0.6]],
    [[0.0, 0.6], [0.0, 0.6]],
    [[0.0, 0.6], [0.0, 0.6]],
    [[0.0, 0.6], [0.0, 0.6]],
    [[0.0, 0.6], [0.0, 0.6]],
    [[0.0, 0.6], [0.0, 0.6]]])

In [4]:
# t_mat = np.array([[0.04,-0.2,1],[0.01,-0.1,1],[0,0,1]])
t_mat0 = np.array([
    [1,0.0,0.0],
    [1,0.1,0.01],
    [1,0.2,0.04]])
t_mat1 = np.array([
    [1,-0.1,0.01],
    [1,0.0,0.0],
    [1,0.1,0.01]])
t_mat2 = np.array([
    [1,-0.2,0.04],
    [1,-0.1,0.01],
    [1,0.0,0.0]])
t_mat_inv0 = np.linalg.inv(t_mat0)
t_mat_inv1 = np.linalg.inv(t_mat1)
t_mat_inv2 = np.linalg.inv(t_mat2)

In [5]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations[:, None, -8:]    # get the current state (for each agent)
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
offset = np.array([0, 0.0981, 0])
i_delay = 6
for ii in range(i_delay + 6):                                      # play game for 5 episodes
    j = 0
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    if ii < i_delay:
        continue
    i = ii - i_delay
    states_next = env_info.vector_observations        # get the current state (for each agent)
    states = states_next[:, None, -8:]                # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    states_next = states_next.reshape((2, 3, 8))
    abc_racket0 = t_mat_inv0 @ states_next[0, :, 1] + offset
    abc_racket1 = t_mat_inv1 @ states_next[0, :, 1] + offset
    abc_racket2 = t_mat_inv2 @ states_next[0, :, 1] + offset
#     abc_ball1 = t_mat_inv1 @ states_next[0, :, 5]
#     print(abc_racket1, abc_ball1)
#     abc_ball2 = t_mat_inv2 @ states_next[0, :, 5] + offset
#     print(abc_racket2, abc_ball2)
    print(f'{j:>2} {abc_racket0[0]:>9.4f} {abc_racket0[1]:>9.4f} {abc_racket0[2]:>9.4f}', end='|')
    print(f'{abc_racket1[0]:>9.4f} {abc_racket1[1]:>9.4f} {abc_racket1[2]:>9.4f}', end='|')
    print(f'{abc_racket2[0]:>9.4f} {abc_racket2[1]:>9.4f} {abc_racket2[2]:>9.4f}')

    while True:
        j += 1
        if j > 10:
            actions = test_actions[i]
        else:
            actions = np.array([[0,0],[0,0]])
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        states_next = env_info.vector_observations
        states = np.hstack([states, states_next[:, None, -8:]])
        if np.any(env_info.local_done):                    # exit loop if episode finished
            break
        states_next = states_next.reshape((2, 3, 8))

        abc_racket0 = t_mat_inv0 @ states_next[0, :, 1] + offset
        abc_racket1 = t_mat_inv1 @ states_next[0, :, 1] + offset
        abc_racket2 = t_mat_inv2 @ states_next[0, :, 1] + offset
#         abc_ball1 = t_mat_inv1 @ states_next[0, :, 5]
#         print(abc_racket1, abc_ball1)
#         abc_ball2 = t_mat_inv2 @ states_next[0, :, 5] + offset
#         print(abc_racket2, abc_ball2)
        print(f'{j:>2} {abc_racket0[0]:>9.4f} {abc_racket0[1]:>9.4f} {abc_racket0[2]:>9.4f}', end='|')
        print(f'{abc_racket1[0]:>9.4f} {abc_racket1[1]:>9.4f} {abc_racket1[2]:>9.4f}', end='|')
        print(f'{abc_racket2[0]:>9.4f} {abc_racket2[1]:>9.4f} {abc_racket2[2]:>9.4f}')
    print_obs(states, actions)

env.close()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


 0    0.0000    7.5981  -75.0000|   0.0000   -7.4019  -75.0000|  -1.5000  -22.4019  -75.0000
 1    0.0000  -22.1076   72.0570|  -1.5000   -7.6962   72.0570|  -1.5589    6.7152   72.0570
 2   -1.5000   -0.0000   -4.9050|  -1.5589   -0.9810   -4.9050|  -1.7158   -1.9620   -4.9050
 3   -1.5589   -1.5736    1.0210|  -1.7158   -1.3694    1.0210|  -1.8524   -1.1652    1.0210
 4   -1.7158   -1.9510    6.8368|  -1.8524   -0.5836    6.8368|  -1.8522    0.7837    6.8368
 5   -1.8524    0.1011   -0.0098|  -1.8522    0.0991   -0.0098|  -1.8522    0.0971   -0.0098
 6   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 7   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 8   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 9   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
10   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.85

 2   -1.5000   -0.0000   -4.9050|  -1.5589   -0.9810   -4.9050|  -1.7158   -1.9620   -4.9050
 3   -1.5589   -1.5736    1.0211|  -1.7158   -1.3694    1.0211|  -1.8524   -1.1652    1.0211
 4   -1.7158   -1.9510    6.8368|  -1.8524   -0.5836    6.8368|  -1.8522    0.7837    6.8368
 5   -1.8524    0.1011   -0.0098|  -1.8522    0.0991   -0.0098|  -1.8522    0.0971   -0.0098
 6   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 7   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 8   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 9   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
10   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
11   -1.8522   -3.2449   33.4304|  -1.8522    3.4411   33.4304|  -1.1836   10.1272   33.4304
12   -1.8522    7.2158   -4.3164|  -1.1836    6.3525   -4.3164|  -0.60

 5   -1.8524    0.1011   -0.0098|  -1.8522    0.0991   -0.0098|  -1.8522    0.0971   -0.0098
 6   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 7   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 8   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 9   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
10   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
11   -1.8522   -3.2449   33.4304|  -1.8522    3.4411   33.4304|  -1.1836   10.1272   33.4304
12   -1.8522    7.2158   -4.3164|  -1.1836    6.3525   -4.3164|  -0.6013    5.4893   -4.3164
13   -1.1836    6.4114   -4.9050|  -0.6013    5.4304   -4.9050|  -0.1171    4.4494   -4.9050
[0.  0.6]
 Right 1    racket   racket   racket   racket |      ball     ball     ball     ball
                 x        y       vx       vy |         x        y  

       6   -7.9878  -1.8522  -0.0000   0.0000 |   -7.1174   3.9281  -0.0000   0.0000
       7   -7.9878  -1.8522  -0.0000   0.0000 |   -7.1174   3.2414  -0.0000   0.0000
       8   -7.9878  -1.8522  -0.0000   0.0000 |   -7.1174   2.4566  -0.0000   0.0000
       9   -7.9878  -1.8522  -0.0000   0.0000 |   -7.1174   1.5742  -0.0000   0.0000
      10   -7.9878  -1.8522  -0.0000   0.0000 |   -7.1174   0.6546  -0.0000   0.0000
      11   -7.9878  -1.1836  -0.0000   6.4114 |   -7.1174  -0.2650  -0.0000   6.4114
      12   -8.0318  -0.7269  -0.0000   2.2895 |   -6.7028  -0.5185  -0.0000   2.2895
      13   -8.0318  -0.5568  -0.0000   1.3085 |   -5.8028   0.1586  -0.0000   1.3085
      14   -8.0318  -0.4848  -0.0000   0.3275 |   -4.9028   0.7375  -0.0000   0.3275
      15   -8.0318  -0.5109  -0.0000  -0.6535 |   -4.0028   1.2184  -0.0000  -0.6535
      16   -8.0318  -0.6352  -0.0000  -1.6345 |   -3.1028   1.6011  -0.0000  -1.6345
      17   -8.0318  -0.8575  -0.0000  -2.6155 |   -2.2028   1.885

 2   -1.5000   -0.0000   -4.9050|  -1.5589   -0.9810   -4.9050|  -1.7158   -1.9620   -4.9050
 3   -1.5589   -1.5736    1.0211|  -1.7158   -1.3694    1.0211|  -1.8524   -1.1652    1.0211
 4   -1.7158   -1.9510    6.8368|  -1.8524   -0.5836    6.8368|  -1.8522    0.7837    6.8368
 5   -1.8524    0.1011   -0.0098|  -1.8522    0.0991   -0.0098|  -1.8522    0.0971   -0.0098
 6   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 7   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 8   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 9   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
10   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
11   -1.8522   -3.2449   33.4304|  -1.8522    3.4411   33.4304|  -1.1836   10.1272   33.4304
12   -1.8522    7.2158   -4.3164|  -1.1836    6.3525   -4.3164|  -0.60

 1    0.0000  -22.1076   72.0570|  -1.5000   -7.6962   72.0570|  -1.5589    6.7152   72.0570
 2   -1.5000   -0.0000   -4.9050|  -1.5589   -0.9810   -4.9050|  -1.7158   -1.9620   -4.9050
 3   -1.5589   -1.5736    1.0210|  -1.7158   -1.3694    1.0210|  -1.8524   -1.1652    1.0210
 4   -1.7158   -1.9510    6.8368|  -1.8524   -0.5836    6.8368|  -1.8522    0.7838    6.8368
 5   -1.8524    0.1011   -0.0098|  -1.8522    0.0991   -0.0098|  -1.8522    0.0971   -0.0098
 6   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 7   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 8   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
 9   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
10   -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000|  -1.8522    0.0981    0.0000
11   -1.8522   -3.2449   33.4304|  -1.8522    3.4411   33.4304|  -1.18

In [6]:
states_next

array([[[-4.68782616e+00,  2.28938133e-01,  3.00000000e+00,
         -3.59479666e+00, -7.11741829e+00, -6.32854819e-01,
          3.00000000e+00, -3.59479666e+00],
        [-4.38782644e+00, -1.89401507e-01,  3.00000000e+00,
         -4.57579613e+00, -7.11741829e+00, -1.55247474e+00,
          3.00000000e+00, -4.57579613e+00],
        [-4.08782673e+00, -7.05841184e-01,  3.00000000e+00,
         -5.55679560e+00, -7.11741829e+00, -2.47209454e+00,
          3.00000000e+00, -5.55679560e+00]],

       [[-1.05888815e+01,  2.28938133e-01, -3.00000000e+00,
         -3.59479666e+00,  7.11741829e+00, -6.32854819e-01,
         -3.00000000e+00, -3.59479666e+00],
        [-1.08888836e+01, -1.89401507e-01, -3.00000000e+00,
         -4.57579613e+00,  7.11741829e+00, -1.55247474e+00,
         -3.00000000e+00, -4.57579613e+00],
        [-1.08997869e+01, -7.05841184e-01, -6.96182251e-04,
         -5.55679560e+00,  7.11741829e+00, -2.47209454e+00,
         -6.96182251e-04, -5.55679560e+00]]])

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```python
env = UnityEnvironment(file_name="Tennis.app")
```

In [3]:
env = UnityEnvironment(file_name="./Tennis_Linux_NoVis/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [6]:
def print_obs0(vec_obs):
    vec_obs = vec_obs.reshape((2, 3, 8))
    for obs in vec_obs:
        for i in range(len(obs)):
            print(' '.join(map(lambda x: f"{x:>8.4f}", obs[i])))
        print('')

In [8]:
print_obs0(states_next)

-10.4573   0.2289  -5.1033  -3.5948  -7.1174  -0.0811  -5.1033  -3.5948
 -7.9763  -0.1894  24.8108  -4.5758  -7.1174  -1.0007  24.8108  -4.5758
 -5.1555  -0.1698  30.0000   0.7170  -5.4417  -2.5895  30.0000   0.7170

 -9.9916   0.6955 -30.0000  -1.6328   7.1174  -0.0811 -30.0000  -1.6328
-10.7589   0.4734  -7.6732  -2.6138   7.1174  -1.0007  -7.6732  -2.6138
-10.9001   0.1531   0.0000  -3.5948   5.4417  -2.5895   0.0000  -3.5948



In [ ]:
env_info = env.reset(train_mode=False)[brain_name]         # reset the environment    
for i in range(1, 6):                                      # play game for 5 episodes
    states = env_info.vector_observations[:, None, -8:]    # get the current state (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        if np.any(env_info.local_done):                    # exit loop if episode finished
            break
        states = np.hstack([states, env_info.vector_observations[:, None, -8:]])
    print_obs(states)


In [18]:
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations[:, None, -8:]
while True:
    actions = np.random.randn(num_agents, action_size)
    actions = np.clip(actions, -1, 1)
    env_info = env.step(actions)[brain_name]
    if np.any(env_info.local_done):
        break
    states_next = env_info.vector_observations
    states = np.hstack([states, states_next[:, None, -8:]])
    states_next = states_next.reshape((2, 3, 8))
    
    print(t_mat_inv @ states_next[0, :, 0], t_mat_inv @ states_next[0, :, 4])
#     print(t_mat_inv @ states_next[1, :, 1], t_mat_inv @ states_next[1, :, 5])

[ 334.32719707 -105.3862977     0.        ] [-352.5881052   105.77643156    0.        ]
[-23.04270267  -2.78386831  -7.1953578 ] [0.        0.        7.0517621]
[198.48341942 -29.54502106  -7.70417166] [0.        0.        7.0517621]
[-5.96046448e-05  3.00000107e+01 -8.67383957e+00] [0.        0.        7.0517621]
[-123.32891226   42.33288407   -5.67383909] [0.        0.        7.0517621]
[60.34988761 -0.70077837 -2.67383981] [0.        0.        7.0517621]
[-87.02093959  26.10628188  -2.14041877] [0.        0.        7.0517621]
[-179.99998927   17.99999893   -0.39999998] [0.        0.        7.0517621]
[121.70236707 -48.17023456  -0.39999998] [0.        0.        7.0517621]
[117.23134518 -23.38265896  -3.99999976] [0.        0.        7.0517621]
[91.06625617  2.68011898 -5.16595221] [-37.97225952   3.79722595   7.0517621 ]
[-257.46468902   55.74646473   -3.98727775] [-7.02769756 -6.89168215  7.0517621 ]
[-42.53526628 -17.23941535  -0.98727816] [ 0.         -8.99999142  6.29231691]
[  

In [19]:
print_obs(states)

 Right 1    racket   racket   racket   racket |      ball     ball     ball     ball
                 x        y       vx       vy |         x        y       vx       vy
       0   -7.1954  -1.5000  -0.0000   0.0000 |    7.0518   5.9882  -0.0000   0.0000
       1   -7.7042  -1.5589  -5.0881  -0.9810 |    7.0518   5.8901  -5.0881  -0.9810
       2   -8.6738  -0.9177  -9.6967   6.0190 |    7.0518   5.6939  -9.6967   6.0190
       3   -5.6738  -0.3747  30.0000   5.0380 |    7.0518   5.3996  30.0000   5.0380
       4   -2.6738   0.0703  30.0000   4.0570 |    7.0518   5.0072  30.0000   4.0570
       5   -2.1404   0.4171   5.3342   3.0760 |    7.0518   4.5167   5.3342   3.0760
       6   -0.4000   0.6658  30.0000   2.0950 |    7.0518   3.9281  30.0000   2.0950
       7   -0.4000   0.8165  30.0000   1.1140 |    7.0518   3.2414  30.0000   1.1140
       8   -4.0000   0.8690 -30.0000   0.1330 |    7.0518   2.4566 -30.0000   0.1330
       9   -5.1660   0.8235 -11.6595  -0.8480 |    7.0518   1.574

In [ ]:
env.close()

In [9]:
env.close()

### 3. Train the Agents!

Now train our own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can load the saved model weights to watch the agents using the appropriate visualization Simulator above! 

In [ ]:
def ddpg(scores, n_episodes=2000, t_max=1000, print_every=100):
    """Deep Deterministic Policy Gradients.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        t_max (int): maximum number of timesteps per episode
        print_every (int): print after this many episodes. Also used to define length of the deque buffer.
    """
    
    scores_window = deque(maxlen=print_every)  # last 100 scores
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        agent.reset()
        episode_scores = np.zeros(num_agents)                  # initialize the score (for each agent)
        t_step = 0
        while True:
            
            actions = agent.act(states) # based on the current states generate actions.

            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            states_next = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episodes finished

            agent.step(states, actions, rewards, states_next, dones) # agent executes a step and learns
            
            states = states_next                               # roll over states to next time step
            episode_scores += rewards                          # update the score (for each agent)
            
            if np.any(dones):                                  # exit loop if episode finished
                break
                
            t_step += 1 # increment the number of steps seen this episode.
            if t_step >= t_max: # exit loop if episode finished
                episode_scores = episode_scores * 1000.0 / t_step
                break
        
        scores.append(np.mean(episode_scores))
        scores_window.append(np.mean(episode_scores))  # save most recent score
        print('\rEpisode {}\tCurrent Score: {:.2f}\tAverage Score: {:.2f}'.format(i_episode, scores[-1], np.mean(scores_window)), end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tCurrent Score: {:.2f}\tAverage Score: {:.2f}'.format(i_episode, scores[-1], np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

        if np.mean(scores_window) >= 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break

In [10]:
from ddpg_agent import Agent

agent = Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=0)

ModuleNotFoundError: No module named 'ddpg_agent'

In [ ]:
# Train
scores = []
t0 = time.time()
ddpg(scores, n_episodes=3000, t_max=1000)
print(time.time() - t0, 'seconds')

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores) + 1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

When finished, you can close the environment.

### 4: Test the Agents (Optional)

Check that the agents learned by visualizing them in the Tennis Simulator.

In [ ]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]
print('Number of agents:', num_agents)
print('Size of each action:', action_size)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=0)

agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

In [ ]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))